In [1]:
ls

In [1]:
# -*- coding: utf-8 -*-

 

import cv2

import numpy as np
import os

In [2]:
# 读取图片

imagePath = './cv.png'
if os.path.exists(imagePath):
    img = cv2.imread(imagePath)
else:
    print('err','路径不存在')

In [3]:
img.shape

(276, 296, 3)

In [4]:
# 转化成灰度图
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)



In [5]:

# cv2.imshow('img', img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [6]:

# 利用Sobel边缘检测生成二值图

sobel = cv2.Sobel(gray, cv2.CV_8U, 1, 0,ksize=3)

In [7]:
# 二值化

ret, binary = cv2.threshold(sobel, 0, 255,cv2.THRESH_OTSU + cv2.THRESH_BINARY)


In [8]:
# 膨胀、腐蚀

element1 = cv2.getStructuringElement(cv2.MORPH_RECT,(30, 9))

element2 =cv2.getStructuringElement(cv2.MORPH_RECT, (24, 6))

In [9]:
# 膨胀一次，让轮廓突出

dilation = cv2.dilate(binary, element2,iterations=1)

 

# 腐蚀一次，去掉细节

erosion = cv2.erode(dilation, element1,iterations=1)

 

# 再次膨胀，让轮廓明显一些

dilation2 = cv2.dilate(erosion, element2,iterations=2)

In [13]:

#  查找轮廓和筛选文字区域

region = []

binary,contours, hierarchy =cv2.findContours(dilation2, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

for i in range(len(contours)):

   cnt = contours[i]

 

    #计算轮廓面积，并筛选掉面积小的

   area = cv2.contourArea(cnt)

   if (area < 1000):

       continue

 

    #找到最小的矩形

   rect = cv2.minAreaRect(cnt)

   print ("rect is: ")

   print (rect)

 

    #box是四个点的坐标

   box = cv2.boxPoints(rect)
   print('0',np.array(box).shape)

   box = np.int0(box)
   print('1',np.array(box).shape)

 

    #计算高和宽

   height = abs(box[0][1] - box[2][1])

   width = abs(box[0][0] - box[2][0])

 

    #根据文字特征，筛选那些太细的矩形，留下扁的

   if (height > width * 1.3):

       continue

 

   region.append(box)


rect is: 
((147.5, 137.5), (295.0, 275.0), -0.0)
0 (4, 2)
1 (4, 2)
rect is: 
((160.0, 22.5), (72.0, 23.0), -0.0)
0 (4, 2)
1 (4, 2)


In [13]:

# 绘制轮廓

for box in region:

   cv2.drawContours(img, [box], 0, (0, 255, 0), 2)

In [12]:

for box in region:
    print(np.array(box))

[[  0 275]
 [  0   0]
 [295   0]
 [295 275]]
[[124  34]
 [124  11]
 [196  11]
 [196  34]]


In [16]:
box[1]

array([124,  11], dtype=int64)

In [17]:
box[3]

array([196,  34], dtype=int64)

In [ ]:



 

# cv2.imshow('img', img)

# cv2.waitKey(0)

# cv2.destroyAllWindows()

In [3]:
# 读取图片

# img = cv2.imread(imagePath)
 
# 灰度化
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
vis = img.copy()
orig = img.copy()
 
# 调用 MSER 算法
mser = cv2.MSER_create()
regions, _ = mser.detectRegions(gray)  # 获取文本区域
hulls = [cv2.convexHull(p.reshape(-1, 1,2)) for p in regions]  # 绘制文本区域
cv2.polylines(img, hulls, 1, (0, 255, 0))
cv2.imshow('img', img)
 
# 将不规则检测框处理成矩形框
keep = []
for c in hulls:
   x, y, w, h = cv2.boundingRect(c)
   keep.append([x, y, x + w, y + h])
   cv2.rectangle(vis, (x, y), (x + w, y + h), (255, 255, 0), 1)
cv2.imshow("hulls",vis)